In [2]:
!pip install tensorflow-text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.8 MB/s 
     |████████████████████████████████| 588.3 MB 19 kB/s 
     |████████████████████████████████| 6.0 MB 36.2 MB/s 
     |████████████████████████████████| 1.7 MB 48.2 MB/s 
     |████████████████████████████████| 439 kB 61.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/data_for_colab/tripadvisor_hotel_reviews.csv')

df = df[df.Rating != 3]

df['is_positive'] = (df['Rating'] >= 4).astype(int)

Mounted at /content/drive


In [5]:
df.drop(columns=['Rating'], inplace=True)

In [6]:
df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
df

,DATA_COLUMN,LABEL_COLUMN
0,nice hotel expensive parking got good deal sta...,1
1,ok nothing special charge diamond member hilto...,0
3,"unique, great stay, wonderful time hotel monac...",1
4,"great stay great stay, went seahawk game aweso...",1
5,love monaco staff husband stayed hotel crazy w...,1
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",1
20487,great location price view hotel great quick pl...,1
20488,"ok just looks nice modern outside, desk staff ...",0
20489,hotel theft ruined vacation hotel opened sept ...,0


In [7]:
df['LABEL_COLUMN'].value_counts()

1    15093
0     3214
Name: LABEL_COLUMN, dtype: int64

In [14]:
df_positive = df[df['LABEL_COLUMN']==1]

In [15]:
df_negative = df[df['LABEL_COLUMN']==0]

In [36]:
# Тестовая выборка

In [37]:
n_test = (df_negative.shape[0] // 4) * 3
df_negative_test = df_negative.tail(n_test)
n_test = (df_positive.shape[0] // 20) * 3
df_positive_test = df_positive.tail(n_test)

In [38]:
df_positive_test

,DATA_COLUMN,LABEL_COLUMN
17565,wonderful hotel wonderful town unbeknownst man...,1
17566,wonderful wonderful wonderful just returned ye...,1
17567,"affinia 50 you'all, stayed affinia 50 week, li...",1
17568,just right good location walking distance bloo...,1
17569,loved place want thank outstanding service goi...,1
...,...,...
20480,great play stay stay loyal inn package deal ha...,1
20482,great choice wife chose best western quite bit...,1
20483,good bed clean convenient just night happy sta...,1
20486,"best kept secret 3rd time staying charm, not 5...",1


In [39]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [40]:
df_balanced_test.sample(10)

,DATA_COLUMN,LABEL_COLUMN
18864,fabulous efficiency fantastic stay possible fa...,1
19018,construction noise nightmare eden rembrandt sq...,0
6572,"wo n't planned trip group 11 including, booked...",0
18274,excellent hotel not disagree previous reviewer...,1
20034,"great value fell love beach, travelled boyfrie...",1
19911,"best hotel experience, star option choose, fam...",1
17566,wonderful wonderful wonderful just returned ye...,1
18471,"perfectly perfect stay, wow, hotel just perfec...",1
19757,"n't better hotel grace simply best, fiance wen...",1
17977,not picky people girlfriend just returned 6 da...,1


In [41]:
df_balanced_test['LABEL_COLUMN'].value_counts()

0    2409
1    2262
Name: LABEL_COLUMN, dtype: int64

In [42]:
# Обучающая выборка

In [43]:
n_train = df_negative.shape[0] // 4
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 20
df_positive_train = df_positive.head(n_train)

In [44]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [45]:
df_balanced_train['LABEL_COLUMN'].value_counts()

0    803
1    754
Name: LABEL_COLUMN, dtype: int64

In [46]:
df_balanced_train.sample(10)

,DATA_COLUMN,LABEL_COLUMN
713,"great hotel, told desk staff write version sta...",1
2553,nice place eye credit card just came 05/28/200...,0
2728,"usa stay away, family stayed bavaro beach hote...",0
643,"fairmont gold level try, husband looking neede...",1
2015,place bad place bad huge cochroch pillow sheet...,0
1942,fortune fortune house fortune house condo hote...,0
3012,"traveler beware, traveled couple celebrate 40t...",0
371,"enjoyable vintage park boutique hotel, emphasi...",1
3340,surely stay stayed executive-level room newer ...,0
1517,"scorpion stings, quite disappointed arrival re...",0


In [47]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['LABEL_COLUMN'].squeeze()

In [48]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['LABEL_COLUMN'].squeeze()

In [49]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")


In [50]:
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [51]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [52]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [53]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [54]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [55]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [56]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
49/49 [==============================] - 714s 14s/step - loss: 0.6862 - accuracy: 0.5453 - precision: 0.5311 - recall: 0.5212
Epoch 2/10
49/49 [==============================] - 697s 14s/step - loss: 0.6357 - accuracy: 0.6525 - precision: 0.6490 - recall: 0.6154
Epoch 3/10
49/49 [==============================] - 703s 14s/step - loss: 0.6024 - accuracy: 0.6795 - precision: 0.6819 - recall: 0.6340
Epoch 4/10
49/49 [==============================] - 692s 14s/step - loss: 0.5700 - accuracy: 0.7322 - precision: 0.7280 - recall: 0.7135
Epoch 5/10
49/49 [==============================] - 692s 14s/step - loss: 0.5466 - accuracy: 0.7656 - precision: 0.7510 - recall: 0.7719
Epoch 6/10
49/49 [==============================] - 693s 14s/step - loss: 0.5204 - accuracy: 0.7900 - precision: 0.7897 - recall: 0.7719
Epoch 7/10
49/49 [==============================] - 696s 14s/step - loss: 0.5111 - accuracy: 0.7694 - precision: 0.7680 - recall: 0.7507
Epoch 8/10
49/49 [=======================

In [57]:
y_predicted = model.predict(X_test)
y_predicted


146/146 [==============================] - 2101s 14s/step


array([[0.56682944],
       [0.52013457],
       [0.19184993],
       ...,
       [0.44941565],
       [0.65389407],
       [0.83611906]], dtype=float32)

In [60]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([1, 1, 0, ..., 0, 1, 1])

In [61]:
y_test

4133     0
4134     0
4135     0
4136     0
4140     0
        ..
20480    1
20482    1
20483    1
20486    1
20487    1
Name: LABEL_COLUMN, Length: 4671, dtype: int64

In [62]:
y_test_array = np.array(y_test)

In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predicted)

0.7467351744808393

In [64]:
from sklearn.metrics import precision_score
precision_score(y_test_array, y_predicted)


0.7064676616915423

In [65]:
from sklearn.metrics import recall_score
recall_score(y_test_array, y_predicted)

0.8160919540229885

In [66]:
from sklearn.metrics import f1_score
f1_score(y_test_array, y_predicted)

0.7573333333333334

In [67]:
df_results_for_reviews_on_hotels = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [68]:
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'precision'] = precision_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'recall'] = recall_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'f1_score'] =  f1_score(y_test_array, y_predicted)

In [ ]:
# trained on hotels

In [70]:
df_results_for_reviews_on_hotels

,accuracy,precision,recall,f1_score
reviews on hotels,0.746735,0.706468,0.816092,0.757333


In [71]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/bert_trained_on_hotel_reviews_11_december'

In [72]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/bert_trained_on_hotel_reviews_11_december


In [73]:
model.save(saved_model_path, include_optimizer=True) 